In [2]:
import numpy as np

In [3]:
matriz_nodos=np.array([[0,0],[3,0],[6,0],[6,-3]])   #cada fila tiene la posición del i-esimo nodo
matriz_conectividad=np.array([[0,1],[1,2],[2,3]]) #cada fila dice entre que nodos se encuentra el i-esimo elemento
gr_libertad=np.array([[0,0],[0,1],[1,1],[0,0]]) #grados de libertad de cada nodo (2 por nodo, el primero es el gr libertad en x, el segundo en y). 0=restringido, 1=libre
n=len(matriz_nodos)
matriz_global=np.zeros((2*n,2*n))
fuerzas_dato=np.array([0,-50e3,0]) #Cada fila hace referencia a cada nodo, la columna 0 son las proyecciones en x de la fza y la columna 1, las proyecciones en y
delta=np.array([0,0,0,0,0])
E=210e9


r=([0,1,2,6,7]) #nodos vinculados
s=([3,4,5])
tipo=np.array([[0],[0],[1]]) 
I=2e-4
cte_resorte=200e3
np.set_printoptions(precision = 4, linewidth = 132)
#scale=np.max((np.max(matriz_global)))
#matriz_global[abs(matriz_global/scale) < 1e-9] = 0


In [6]:
for i in range (len(matriz_conectividad)):
    (a),(b)=matriz_conectividad[i] 
    a=int(a)
    b=int(b)
    x1=matriz_nodos[a,0]
    x2=matriz_nodos[b,0]
    y1=matriz_nodos[a,1]
    y2=matriz_nodos[b,1]

    angulo=np.arctan2((y2-y1),(x2-x1)) #angulo entre los elementos y el eje de coordenadas
    #print(angulo*180/np.pi)
    S=np.sin(angulo)
    C=np.cos(angulo)
    L=np.sqrt(((y2-y1)**2)+((x2-x1)**2)) #Largo del elemento
   
    if tipo[i]==0:
        coef_k=E*I/L**3
        k=coef_k*np.array([[12,6*L,-12,6*L],[6*L,4*L**2,-6*L,2*L**2],[-12,-6*L,12,-6*L],[6*L,2*L**2,-6*L,4*L**2]])
        matriz_global[2*a:2*(a+1), 2*a:2*(a+1)] += k [0:2,0:2]
        matriz_global[2*(a):2*(a+1), 2*(b):2*(b+1)] += k [0:2,2:4]
        matriz_global[2*b:2*(b+1), 2*b:2*(b+1)] += k [2:4,2:4]
        matriz_global[2*b:2*(b+1), 2*a:2*(a+1)] += k [2:4,0:2]
    else:
        coef_k=cte_resorte
        k=coef_k*np.array([[1,0,-1,0],[0,0,0,0],[-1,0,1,0],[0,0,0,0]])
        matriz_global[2*a:2*(a+1), 2*a:2*(a+1)] += k [0:2,0:2]
        matriz_global[2*(a):2*(a+1), 2*(b):2*(b+1)] += k [0:2,2:4]
        matriz_global[2*b:2*(b+1), 2*b:2*(b+1)] += k [2:4,2:4]
        matriz_global[2*b:2*(b+1), 2*a:2*(a+1)] += k [2:4,0:2]
scale=np.max((np.max(matriz_global)))
matriz_global[abs(matriz_global/scale) < 1e-9] = 0
print(matriz_global)

[[ 3.7333e+07  5.6000e+07 -3.7333e+07  5.6000e+07  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00]
 [ 5.6000e+07  1.1200e+08 -5.6000e+07  5.6000e+07  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00]
 [-3.7333e+07 -5.6000e+07  7.4667e+07  0.0000e+00 -3.7333e+07  5.6000e+07  0.0000e+00  0.0000e+00]
 [ 5.6000e+07  5.6000e+07  0.0000e+00  2.2400e+08 -5.6000e+07  5.6000e+07  0.0000e+00  0.0000e+00]
 [ 0.0000e+00  0.0000e+00 -3.7333e+07 -5.6000e+07  3.7733e+07 -5.6000e+07 -4.0000e+05  0.0000e+00]
 [ 0.0000e+00  0.0000e+00  5.6000e+07  5.6000e+07 -5.6000e+07  1.1200e+08  0.0000e+00  0.0000e+00]
 [ 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 -4.0000e+05  0.0000e+00  4.0000e+05  0.0000e+00]
 [ 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00]]


In [7]:
k_reducida=matriz_global[np.ix_(s,s)]
k_vinculo=matriz_global[np.ix_(s,r)]
print(k_reducida, k_vinculo)

[[ 2.2400e+08 -5.6000e+07  5.6000e+07]
 [-5.6000e+07  3.7733e+07 -5.6000e+07]
 [ 5.6000e+07 -5.6000e+07  1.1200e+08]] [[ 56000000.      56000000.             0.             0.             0.    ]
 [        0.             0.     -37333333.3333   -400000.             0.    ]
 [        0.             0.      56000000.             0.             0.    ]]


In [8]:
aux=np.linalg.inv(k_reducida)
print(aux)
f=[]
d=[]
for i in range (len(s)):
    f.append(fuerzas_dato[i])
for i in range (len(r)):
    d.append(delta[i])
#print(f)
#print(d)
u=np.dot(aux,f-(np.dot(k_vinculo,d)))
print(u)

[[8.6616e-09 2.4917e-08 8.1277e-09]
 [2.4917e-08 1.7442e-07 7.4751e-08]
 [8.1277e-09 7.4751e-08 4.2240e-08]]
[-0.0012 -0.0087 -0.0037]


In [9]:
desplazamientos=np.zeros((2*n))
#print(desplazamientos)
a=0
for i in (s):
    desplazamientos[i]=u[a]
    a+=1
a=0
for i in ((r)):
    desplazamientos[i]=delta[a]
    a+=1
print(desplazamientos)

[ 0.      0.      0.     -0.0012 -0.0087 -0.0037  0.      0.    ]


In [10]:
aux=matriz_global[np.ix_(r,np.arange(0,2*n))]
print(aux)
for i in r:
    F=np.dot(aux,desplazamientos)
print(F)

[[ 3.7333e+07  5.6000e+07 -3.7333e+07  5.6000e+07  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00]
 [ 5.6000e+07  1.1200e+08 -5.6000e+07  5.6000e+07  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00]
 [-3.7333e+07 -5.6000e+07  7.4667e+07  0.0000e+00 -3.7333e+07  5.6000e+07  0.0000e+00  0.0000e+00]
 [ 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 -4.0000e+05  0.0000e+00  4.0000e+05  0.0000e+00]
 [ 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00]]
[-69767.4419 -69767.4419 116279.0698   3488.3721      0.    ]


In [11]:
resultado_fuerzas=np.zeros((2*n))
a=0
for i in r:
    resultado_fuerzas[i]=F[a]
    a+=1
a=0
for i in s:
    resultado_fuerzas[i]=fuerzas_dato[a]
    a+=1
print(resultado_fuerzas)
print(desplazamientos)

[-69767.4419 -69767.4419 116279.0698      0.     -50000.          0.       3488.3721      0.    ]
[ 0.      0.      0.     -0.0012 -0.0087 -0.0037  0.      0.    ]
